# Supervised Learning Using PyTorch

In [38]:
from __future__ import print_function
import torch
import numpy as np
import torch.optim
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torch.nn.functional as F
from torch.autograd import Variable
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [39]:
# # open csv file and read data for pytorch
import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# import os
# import time
# import copy
# import random
# import math
# import torch
# import torch.nn as nn
# import torch.optim as optim
# import torch.nn.functional as F
# from torch.utils.data import Dataset, DataLoader

# read data
data = pd.read_csv('dataset_Caso_1_processed.csv')
data = data.dropna()
X,Y = data.iloc[:,0:-1],data.iloc[:,-1]
X


,x1,x2,x3_LCV,x3_QKP,x3_SAT,x3_XJB,x4_MZBER,x4_PQKE,x4_YEQA,x4_ZUQF,...,x7_5,x7_6,x7_8,x8_-7.5,x8_-6.5,x8_-5.5,x8_-4.5,x8_-3.5,x8_-2.5,x8_-1.5
0,-2.247572,-1.670521,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,-2.247572,-1.670521,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,1.025939,1.391524,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,-2.247572,-1.670521,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,-0.692015,-0.035019,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,0.802121,0.880734,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1996,0.227936,0.215753,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1997,1.146373,0.183901,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1998,0.590802,0.479194,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [40]:
# add noise to cathegorial features
p = 1e-5

# add noise for every feature separately
X = X + np.random.normal(0,p,X.shape)
X


,x1,x2,x3_LCV,x3_QKP,x3_SAT,x3_XJB,x4_MZBER,x4_PQKE,x4_YEQA,x4_ZUQF,...,x7_5,x7_6,x7_8,x8_-7.5,x8_-6.5,x8_-5.5,x8_-4.5,x8_-3.5,x8_-2.5,x8_-1.5
0,-2.247571,-1.670513,1.336346e-05,-0.000007,9.999931e-01,3.070558e-07,1.000000,-0.000005,-1.462577e-06,0.000013,...,-9.194131e-06,1.000006e+00,-5.642623e-06,0.000019,0.999997,-0.000017,0.000022,-0.000002,0.000004,-1.600673e-05
1,-2.247578,-1.670526,2.848577e-05,0.000004,9.999918e-01,1.084186e-05,1.000001,-0.000003,2.094801e-06,-0.000002,...,-8.121052e-06,9.999939e-01,8.609777e-06,-0.000003,0.000003,-0.000011,1.000010,-0.000007,-0.000006,1.185335e-05
2,1.025935,1.391526,1.000016e+00,-0.000010,1.201850e-05,1.271726e-06,1.000024,0.000003,8.410662e-06,-0.000008,...,-5.184427e-09,1.879509e-05,-1.394652e-05,0.000012,0.000002,0.000004,-0.000002,0.000023,0.999996,6.143002e-06
3,-2.247572,-1.670521,1.350241e-05,0.000013,9.999891e-01,2.680728e-07,1.000000,0.000008,4.903438e-07,-0.000018,...,1.581487e-05,-1.290875e-06,9.696633e-07,-0.000014,0.000009,0.000008,1.000006,-0.000009,-0.000010,-2.666942e-06
4,-0.692005,-0.035029,2.371110e-06,0.000033,-2.908868e-06,9.999901e-01,0.999980,-0.000007,-2.803982e-06,0.000024,...,9.999890e-01,9.591986e-06,-3.521323e-06,-0.000005,0.000006,-0.000006,1.000005,-0.000002,-0.000011,-1.149327e-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,0.802149,0.880729,2.073757e-06,0.999996,2.357311e-07,-3.997955e-06,1.000014,-0.000004,3.130863e-06,-0.000012,...,-5.535543e-08,6.291697e-06,-1.015886e-05,0.000005,0.000003,-0.000013,-0.000011,-0.000007,1.000000,2.781661e-06
1996,0.227937,0.215761,4.018471e-07,-0.000009,-1.054975e-05,9.999933e-01,0.000001,0.000002,9.999761e-01,0.000007,...,6.330760e-06,1.188480e-05,-1.891222e-05,-0.000009,-0.000011,0.000008,-0.000021,0.000014,1.000009,-9.735054e-06
1997,1.146383,0.183903,2.062651e-05,-0.000007,1.000009e+00,8.974153e-06,1.000004,-0.000004,1.008618e-05,-0.000003,...,-2.970408e-06,6.588313e-06,8.169397e-06,-0.000006,-0.000002,0.999986,0.000025,-0.000011,-0.000018,-8.635924e-07
1998,0.590797,0.479184,1.000018e+00,-0.000008,6.802510e-06,-1.569116e-06,1.000010,-0.000011,-1.523550e-05,-0.000010,...,-2.392276e-06,-3.480053e-06,-1.184995e-05,-0.000018,-0.000010,0.999999,-0.000009,-0.000001,-0.000007,-2.809606e-06


In [41]:
# transform data into a tensor
x = torch.tensor(X.values,dtype=torch.float32)
y = torch.tensor(Y.values,dtype=torch.float32).reshape(-1,1)

In [42]:
class Net(torch.nn.Module):
    def __init__(self, n_feature, n_hidden, n_output):
        super(Net, self).__init__()
        # hidden layer
        self.hidden = torch.nn.Linear(n_feature, n_hidden)
        #output layer
        self.out = torch.nn.Linear(n_hidden, n_output)
        self.sigmoid = nn.Sigmoid() # Output between 0 and 1
    def forward(self, x):
        #activation function for the hidden layer
        x = F.sigmoid(self.hidden(x))
        x = self.out(x)  
        x = self.sigmoid(x)
        # linear output
        return x

In [74]:
net = Net(n_feature=26,n_hidden=32,n_output=1)
optimizer = torch.optim.SGD(net.parameters(),lr=1e-5)
weights = torch.tensor([100, 1])  # Example weights for class 0 and 1
loss_func = torch.nn.BCELoss()


for t in range(100):    
    out = net(x)
    # input x and predict based on x

    loss = loss_func(out, y)     
    # must be (1. nn output, 2. target)
    optimizer.zero_grad()   # clear gradients for next train
    loss.backward()         
    # backpropagation, compute gradients
    optimizer.step()        
    # apply gradients
    if t % 10 == 0:
        # print the loss every 10 iterations
        print(loss)

# Count how much elements doesn't match for every class, 1 or 0

# print the confusion matrix
from sklearn.metrics import confusion_matrix
print('Confusion Matrix')
print(confusion_matrix(y.data.numpy(), predicted.data.numpy()))



tensor(0.6150, grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.6148, grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.6146, grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.6144, grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.6142, grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.6140, grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.6138, grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.6137, grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.6135, grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.6133, grad_fn=<BinaryCrossEntropyBackward0>)
Confusion Matrix
[[1978    0]
 [  22    0]]


In [69]:
sum(sum(correct == True))


1978

In [44]:
class Deep(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Linear(26, 100)
        self.act1 = nn.ReLU()
        self.layer2 = nn.Linear(100, 100)
        self.act2 = nn.ReLU()
        self.layer3 = nn.Linear(100, 100)
        self.act3 = nn.ReLU()
        self.output = nn.Linear(100, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.act1(self.layer1(x))
        x = self.act2(self.layer2(x))
        x = self.act3(self.layer3(x))
        x = self.sigmoid(self.output(x))
        return x

In [47]:
import copy
import numpy as np
import torch.nn as nn
import torch.optim as optim
import tqdm
# import f1 score
from sklearn.metrics import f1_score

def model_train(model, X_train, y_train, X_val, y_val):
    # loss function and optimizer
    # weights = torch.tensor([100, 1])  # Example weights for class 0 and 1
    # loss_func = torch.nn.BCELoss(weights.type(torch.FloatTensor))  
    weights = torch.tensor([100, 1]) 
    loss_fn = nn.BCELoss()  # binary cross entropy
    optimizer = optim.Adam(model.parameters(), lr=0.0001)

    n_epochs = 10   # number of epochs to run
    batch_size = 10  # size of each batch
    batch_start = torch.arange(0, len(X_train), batch_size)

    # Hold the best model
    best_acc = - np.inf   # init to negative infinity
    best_weights = None

    for epoch in range(n_epochs):
        print("Epoch: ",epoch)
        # print(epoch)
        model.train()
        with tqdm.tqdm(batch_start, unit="batch", mininterval=0, disable=True) as bar:
            bar.set_description(f"Epoch {epoch}")
            for start in bar:
                # take a batch
                X_batch = X_train[start:start+batch_size]
                y_batch = y_train[start:start+batch_size]
                # forward pass
                y_pred = model(X_batch)
                loss = loss_fn(y_pred, y_batch)
                # backward pass
                optimizer.zero_grad()
                loss.backward()
                # update weights
                optimizer.step()
                # print progress
                acc = (y_pred.round() == y_batch).float().mean()
                bar.set_postfix(
                    loss=float(loss),
                    acc=float(acc)
                )
        # evaluate accuracy at end of each epoch
        model.eval()
        y_pred = model(X_val)
        acc = (y_pred.round() == y_val).float().mean()
        acc = float(acc)
        #compute f1 score
        f1 = f1_score(y_val.detach().numpy(), y_pred.round().detach().numpy())
        # print f1 score
        print(f'F1 score: {f1}')
        if acc > best_acc:
            best_acc = acc
            best_weights = copy.deepcopy(model.state_dict())
    # restore model and return best accuracy
    model.load_state_dict(best_weights)
    return best_acc

In [48]:
from sklearn.model_selection import StratifiedKFold, train_test_split

# train-test split: Hold out the test set for final model evaluation
# define 5-fold cross validation test harness
kfold = StratifiedKFold(n_splits=5, shuffle=True)
cv_scores = []
for train, test in kfold.split(x, y):
    # create model, train, and get accuracy
    model = Deep()
    acc = model_train(model, x[train], y[train], x[test], y[test])
    print("Accuracy (wide): %.2f" % acc)

    cv_scores.append(acc)

# evaluate the model
acc = np.mean(cv_scores)
std = np.std(cv_scores)
print("Model accuracy: %.2f%% (+/- %.2f%%)" % (acc*100, std*100))

Epoch:  0
F1 score: 0.0
Epoch:  1
F1 score: 0.0
Epoch:  2
F1 score: 0.0
Epoch:  3
F1 score: 0.0
Epoch:  4
F1 score: 0.0
Epoch:  5
F1 score: 0.0
Epoch:  6
F1 score: 0.0
Epoch:  7
F1 score: 0.0
Epoch:  8
F1 score: 0.0
Epoch:  9
F1 score: 0.0
Accuracy (wide): 0.99
Epoch:  0
F1 score: 0.0
Epoch:  1
F1 score: 0.0
Epoch:  2
F1 score: 0.0
Epoch:  3
F1 score: 0.0
Epoch:  4
F1 score: 0.0
Epoch:  5
F1 score: 0.0
Epoch:  6
F1 score: 0.0
Epoch:  7
F1 score: 0.0
Epoch:  8
F1 score: 0.0
Epoch:  9
F1 score: 0.0
Accuracy (wide): 0.99
Epoch:  0
F1 score: 0.0
Epoch:  1
F1 score: 0.0
Epoch:  2
F1 score: 0.0
Epoch:  3
F1 score: 0.0
Epoch:  4
F1 score: 0.0
Epoch:  5
F1 score: 0.0
Epoch:  6
F1 score: 0.0
Epoch:  7
F1 score: 0.0
Epoch:  8
F1 score: 0.0
Epoch:  9
F1 score: 0.0
Accuracy (wide): 0.99
Epoch:  0
F1 score: 0.0
Epoch:  1
F1 score: 0.0
Epoch:  2
F1 score: 0.0
Epoch:  3
F1 score: 0.0
Epoch:  4
F1 score: 0.0
Epoch:  5
F1 score: 0.0
Epoch:  6
F1 score: 0.0
Epoch:  7
F1 score: 0.0
Epoch:  8


KeyboardInterrupt: 